In [1]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, SimpleRNN
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from keras.models import load_model
from google.colab import output

In [2]:
#https://colab.research.google.com/notebooks/pro.ipynb#scrollTo=V1G82GuO-tez
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul  2 08:30:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#https://colab.research.google.com/notebooks/pro.ipynb#scrollTo=V1G82GuO-tez
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
tf.test.gpu_device_name() #/device:GPU:0

'/device:GPU:0'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
set_name='flap_jump'
split = 'PS'
batch_size = 32
labels = 2

set_folder = 'cv_'+set_name+'_'+split
cv_set_folder='cv_sets'
train_set_name = set_name+'_train_set_'+split+'_cv_'
test_set_name = set_name+'_test_set_'+split+'_cv_'
os.chdir("/content/drive/My Drive/promotion/data/"+set_folder)
print(os.getcwd())
sequence_length = 15
np.random.seed(42)
column_names_labels = ['Flattern', 'Hüpfen']
column_names_features = ['Shoulder_x_0', 'Shoulder_y_0', 'Elbow_x_0', 'Elbow_y_0',
                     'RWrist_x_0', 'RWrist_y_0', 'LShoulder_x_0', 'LShoulder_y_0',
                     'LElbow_x_0', 'LElbow_y_0', 'LWrist_x_0', 'LWrist_y_0',
                     'Shoulder_x_1', 'Shoulder_y_1', 'Elbow_x_1', 'Elbow_y_1',
                     'RWrist_x_1', 'RWrist_y_1', 'LShoulder_x_1', 'LShoulder_y_1',
                     'LElbow_x_1', 'LElbow_y_1', 'LWrist_x_1', 'LWrist_y_1',
                     'Shoulder_x_2', 'Shoulder_y_2', 'Elbow_x_2', 'Elbow_y_2',
                     'RWrist_x_2', 'RWrist_y_2', 'LShoulder_x_2', 'LShoulder_y_2',
                     'LElbow_x_2', 'LElbow_y_2', 'LWrist_x_2', 'LWrist_y_2',
                     'Shoulder_x_3', 'Shoulder_y_3', 'Elbow_x_3', 'Elbow_y_3',
                     'RWrist_x_3','RWrist_y_3', 'LShoulder_x_3', 'LShoulder_y_3',
                     'LElbow_x_3', 'LElbow_y_3', 'LWrist_x_3', 'LWrist_y_3',
                     'Shoulder_x_4', 'Shoulder_y_4', 'Elbow_x_4', 'Elbow_y_4',
                     'RWrist_x_4', 'RWrist_y_4', 'LShoulder_x_4', 'LShoulder_y_4',
                     'LElbow_x_4', 'LElbow_y_4', 'LWrist_x_4', 'LWrist_y_4']

/content/drive/My Drive/promotion/data/cv_flap_jump_PS


In [7]:
# load data

num_cv = 3
train_df_pre_list = []
test_df_pre_list = []
x_train_list = []
x_test_list = []
y_train_list = []
y_test_list = []
features_list = []

for x in range(num_cv):
  train = pd.read_csv("/content/drive/My Drive/promotion/data/"+cv_set_folder+'/'+train_set_name+str(x), sep=';')
  test = pd.read_csv("/content/drive/My Drive/promotion/data/"+cv_set_folder+'/'+test_set_name+str(x), sep=';')
  train_df_pre_list.append(train)
  test_df_pre_list.append(test)

def reshape_sets(train_df_pre, test_df_pre, column_names_labels, column_names_drop):
    train_df_labels = train_df_pre.loc[:, column_names_labels]
    test_df_labels = test_df_pre.loc[:, column_names_labels]
    train_df = train_df_pre.loc[:, column_names_features]
    test_df = test_df_pre.loc[:, column_names_features]
    x_train = np.asarray(train_df)
    x_test = np.asarray(test_df)
    y_train = np.asarray(train_df_labels)
    y_test = np.asarray(test_df_labels)
    train_samples = int(x_train.shape[0]/sequence_length)
    features = x_train.shape[1]
    test_samples = int(x_test.shape[0]/sequence_length)
    x_train = x_train.reshape((train_samples,sequence_length,features))
    x_test = x_test.reshape((test_samples,sequence_length,features))
    y_train = y_train.reshape((train_samples,sequence_length,labels))
    y_test = y_test.reshape((test_samples,sequence_length,labels))
    y_train = y_train[:,0,:]
    y_test = y_test[:,0,:]
    print("x_train_shape: "+str(x_train.shape))
    print("x_test_shape: "+str(x_test.shape))
    print("y_train_shape: "+str(y_train.shape))
    print("y_test_shape: "+str(y_test.shape))
    print(column_names_labels[0]+" y_train: "+str(round(y_train[:,0].sum()/y_train[:,0].size*100,2))+" ("+
          str(round(y_train[:,0].sum()/(y_train[:,0].sum()+y_test[:,0].sum()),2))+")")
    print(column_names_labels[1]+" y_train: "+str(round(y_train[:,1].sum()/y_train[:,1].size*100,2))+" ("+
          str(round(y_train[:,1].sum()/(y_train[:,1].sum()+y_test[:,1].sum()),2))+")")
    print(column_names_labels[0]+" y_test: "+str(round(y_test[:,0].sum()/y_test[:,0].size*100,2))+" ("+
          str(round(y_test[:,0].sum()/(y_train[:,0].sum()+y_test[:,0].sum()),2))+")")
    print(column_names_labels[1]+" y_test: "+str(round(y_test[:,1].sum()/y_test[:,1].size*100,2))+" ("+
          str(round(y_test[:,1].sum()/(y_train[:,1].sum()+y_test[:,1].sum()),2))+")")
    return x_train, x_test, y_train, y_test, features

# reshape train/test data
for train, test in zip(train_df_pre_list, test_df_pre_list):
  x_train, x_test, y_train, y_test, features = reshape_sets(train, test
                                    , column_names_labels, column_names_features)
  x_train_list.append(x_train)
  x_test_list.append(x_test)
  y_train_list.append(y_train)
  y_test_list.append(y_test)
  features_list.append(features)


x_train_shape: (1296, 15, 60)
x_test_shape: (564, 15, 60)
y_train_shape: (1296, 2)
y_test_shape: (564, 2)
Flattern y_train: 44.83 (0.71)
Hüpfen y_train: 19.21 (0.64)
Flattern y_test: 42.2 (0.29)
Hüpfen y_test: 25.0 (0.36)
x_train_shape: (1040, 15, 60)
x_test_shape: (820, 15, 60)
y_train_shape: (1040, 2)
y_test_shape: (820, 2)
Flattern y_train: 44.33 (0.56)
Hüpfen y_train: 21.83 (0.58)
Flattern y_test: 43.66 (0.44)
Hüpfen y_test: 19.88 (0.42)
x_train_shape: (1384, 15, 60)
x_test_shape: (476, 15, 60)
y_train_shape: (1384, 2)
y_test_shape: (476, 2)
Flattern y_train: 43.06 (0.73)
Hüpfen y_train: 21.97 (0.78)
Flattern y_test: 46.85 (0.27)
Hüpfen y_test: 18.07 (0.22)


In [8]:
print('test_set: ' + str(x_test_list[0].shape[0]/(x_test_list[0].shape[0]+x_train_list[0].shape[0])*100))

test_set: 30.32258064516129


In [9]:
best_model = load_model('/content/drive/My Drive/promotion/data/'+set_folder+'_tuner/best_model.h5')
best_learning_rate = best_model.optimizer.get_config()['learning_rate']
best_batch_size = batch_size

# iterate over folds
for i, (x_train, x_test, y_train, y_test) in enumerate(zip(x_train_list, x_test_list,
                                                           y_train_list, y_test_list)):
  model_save_name = 'new_model_'+str(i)+'.h5'
  tuner_model = tf.keras.Sequential().from_config(best_model.get_config())
  tuner_model.summary()
  es = EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=100)
  mc = ModelCheckpoint(model_save_name, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
  tuner_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=best_learning_rate), metrics=[keras.metrics.BinaryAccuracy()])
  history = tuner_model.fit(x_train, y_train, epochs=1000, batch_size=best_batch_size, validation_data=(x_test, y_test), callbacks=[es, mc])
  hist_df = pd.DataFrame(history.history)
  new_model = load_model(model_save_name)
  new_model.summary()
  scores_new_model = new_model.evaluate(x_test, y_test, verbose=0)
  results_new_model = {}
  yhat = best_model.predict(x_test, verbose=0)
  yhat_round = yhat.round()
  yhat_flap = yhat_round[:,:1]
  yhat_jump = yhat_round[:,1:2]
  y_test_flap = y_test[:,:1]
  y_test_jump = y_test[:,1:2]
  accuracy_flap = accuracy_score(y_test_flap, yhat_flap)
  print('Accuracy flap: %f' % accuracy_flap)
  results_new_model['accuracy_flap'] = accuracy_flap
  precision_flap = precision_score(y_test_flap, yhat_flap)
  print('Precision flap: %f' % precision_flap)
  results_new_model['precision_flap'] = precision_flap
  recall_flap = recall_score(y_test_flap, yhat_flap)
  print('Recall flap: %f' % recall_flap)
  results_new_model['recall_flap'] = recall_flap
  f1_flap = f1_score(y_test_flap, yhat_flap)
  print('F1 score flap: %f' % f1_flap)
  results_new_model['f1_flap'] = f1_flap
  confusion_matrix_flap = multilabel_confusion_matrix(y_test_flap, yhat_flap)
  accuracy_jump = accuracy_score(y_test_jump, yhat_jump)
  print('Accuracy jump: %f' % accuracy_jump)
  results_new_model['accuracy_jump'] = accuracy_jump
  precision_jump = precision_score(y_test_jump, yhat_jump)
  print('Precision jump: %f' % precision_jump)
  results_new_model['precision_jump'] = precision_jump
  recall_jump = recall_score(y_test_jump, yhat_jump)
  print('Recall jump: %f' % recall_jump)
  results_new_model['recall_jump'] = recall_jump
  f1_jump = f1_score(y_test_jump, yhat_jump)
  print('F1 score jump: %f' % f1_jump)
  results_new_model['f1_jump'] = f1_jump
  confusion_matrix_jump = multilabel_confusion_matrix(y_test_jump, yhat_jump)
  cm = multilabel_confusion_matrix(y_test, yhat_round)
  print(cm)
  cm_flap = cm[0]
  cm_jump = cm[1]
  print(cm_flap)
  print(cm_jump)
  print(classification_report(y_test,yhat_round,target_names=['flap', 'jump']))
  hist_df.to_csv('history_df_'+str(i), sep=';', index=False)
  results_new_model_df = pd.DataFrame(list(results_new_model.items()), columns = ['measure','value'])
  results_new_model_df.to_csv('results_new_model_'+str(i), sep=';', index=False)
  confusion = pd.DataFrame(confusion_matrix_flap[:,0,:])
  confusion.to_csv('confusion_matrix_new_model_flap_'+str(i), sep=';', index=False)
  confusion = pd.DataFrame(confusion_matrix_jump[:,0,:])
  confusion.to_csv('confusion_matrix_new_model_jump'+str(i), sep=';', index=False)
  report = classification_report(y_test,yhat_round,target_names=['flap', 'jump'],output_dict=True)
  report_df = pd.DataFrame(report).transpose()
  report_df.to_csv('classification_report_new_model'+str(i), sep=';', index=False)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 224)           255360    
                                                                 
 lstm_1 (LSTM)               (None, 15, 192)           320256    
                                                                 
 lstm_2 (LSTM)               (None, 224)               373632    
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 2)                 450       
                                                                 
Total params: 949,698
Trainable params: 949,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
37/41 [==========================>...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 224)           255360    
                                                                 
 lstm_1 (LSTM)               (None, 15, 192)           320256    
                                                                 
 lstm_2 (LSTM)               (None, 224)               373632    
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 2)                 450       
                                                                 
Total params: 949,698
Trainable params: 949,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
28/33 [========================>.....

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 224)           255360    
                                                                 
 lstm_1 (LSTM)               (None, 15, 192)           320256    
                                                                 
 lstm_2 (LSTM)               (None, 224)               373632    
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 2)                 450       
                                                                 
Total params: 949,698
Trainable params: 949,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
44/44 [==============================

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [10]:
results_cv = pd.DataFrame(columns=['measure','value'])
results_list = []
for x in range(num_cv):
  results = pd.read_csv('results_new_model_'+str(x), sep=';')
  results_list.append(results)
data = pd.concat(results_list, ignore_index=False)
data_grouped = data.groupby('measure')['value'].mean()
print(data)
print(data_grouped)

          measure     value
0   accuracy_flap  0.854610
1  precision_flap  0.765306
2     recall_flap  0.945378
3         f1_flap  0.845865
4   accuracy_jump  0.911348
5  precision_jump  0.793548
6     recall_jump  0.872340
7         f1_jump  0.831081
0   accuracy_flap  0.619512
1  precision_flap  0.579861
2     recall_flap  0.466480
3         f1_flap  0.517028
4   accuracy_jump  0.693902
5  precision_jump  0.238095
6     recall_jump  0.245399
7         f1_jump  0.241692
0   accuracy_flap  0.804622
1  precision_flap  0.728873
2     recall_flap  0.928251
3         f1_flap  0.816568
4   accuracy_jump  0.936975
5  precision_jump  0.780000
6     recall_jump  0.906977
7         f1_jump  0.838710
measure
accuracy_flap     0.759581
accuracy_jump     0.847408
f1_flap           0.726487
f1_jump           0.637161
precision_flap    0.691347
precision_jump    0.603881
recall_flap       0.780037
recall_jump       0.674905
Name: value, dtype: float64
